In [1]:
from nba_api.stats.endpoints import leaguedashteamstats, scoreboard, leaguegamefinder
from nba_api.stats.static import teams
import requests
import json
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
cavs = 1610612739
clippers = 1610612746
knicks = 1610612752
celtics = 1610612738
hawks = 1610612737
nets = 1610612751
hornets = 1610612766
bulls = 1610612741
mavericks = 1610612742
nuggets = 1610612743
pistons = 1610612765
warriors = 1610612744
rockets = 1610612745
pacers = 1610612754
lakers = 1610612747
grizzlies = 1610612763
heat = 1610612748
bucks = 1610612749
timberwolves = 1610612750
pelicans = 1610612740
thunder = 1610612760
magic = 1610612753
sixers = 1610612755
suns = 1610612756
blazers = 1610612757
kings = 1610612758
spurs = 1610612759
raptors = 1610612761
jazz = 1610612762
wizards = 1610612764

league_ids = [cavs, clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
             pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
             spurs, raptors, jazz, wizards]
no_cavs = [cavs, clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
          pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
          spurs, raptors, jazz, wizards]

In [3]:
def combine_team_games(df, keep_method='home'):
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_AWAY', '_HOME'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_AWAY != joined.TEAM_ID_HOME]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_AWAY.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_AWAY.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_AWAY == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_AWAY == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
#     display(joined)
    return result

In [106]:
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

def get_dfs(team_id):
    seasons = ['2022-23', '2021-22', '2020-21', '2019-20']
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable='2023-24')
    all_team_games = gamefinder.get_data_frames()[0]
    
    for i in seasons:
        new_gf = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=i)
        new_games = new_gf.get_data_frames()[0]
        all_team_games = all_team_games.append(new_games)

    all_team_games.index = range(0, all_team_games.shape[0])
    game_ids = []
    for i in range(all_team_games.shape[0]):
        game_ids.append(all_team_games.loc[i]['GAME_ID'])
        
    rows = []
    for id in game_ids:
        get_games = all_games[all_games.GAME_ID == id]
        new_row = combine_team_games(get_games).iloc[0]
        rows.append(new_row)
    df = pd.DataFrame(rows)
    df_away = df[df['TEAM_ID_AWAY'].isin([team_id])]
    df_home = df[df['TEAM_ID_HOME'].isin([team_id])]
    df_home = df_home.sort_values(by='GAME_DATE', ascending=True)
    df_away = df_away.sort_values(by='GAME_DATE', ascending=True)
    df_away.index = range(0,df_away.shape[0])
    df_home.index = range(0,df_home.shape[0])
    return df_home, df_away

In [107]:
df_home, df_away = get_dfs(knicks)
pd.set_option('max_columns', None)
df_away

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME
0,12019,1610612752,NYK,New York Knicks,0011900041,2019-10-11,NYK vs. WAS,L,242,99,32,82,0.390,10,34,0.294,25,32,0.781,12,40,52,21,9,4,23,28,-16.0,1610612764,WAS,Washington Wizards,WAS @ NYK,W,240,115,40,89,0.449,19,41,0.463,16,23,0.696,7,36,43,23,12,4,18,27,16.0
1,12019,1610612752,NYK,New York Knicks,0011900063,2019-10-16,NYK vs. ATL,L,240,96,32,81,0.395,7,30,0.233,25,33,0.758,6,38,44,22,6,10,14,21,-4.0,1610612737,ATL,Atlanta Hawks,ATL @ NYK,W,240,100,36,98,0.367,10,34,0.294,18,22,0.818,16,42,58,23,9,4,11,22,4.0
2,12019,1610612752,NYK,New York Knicks,0011900077,2019-10-18,NYK vs. NOP,L,239,116,44,93,0.473,13,33,0.394,15,24,0.625,12,31,43,23,5,8,18,23,-1.0,1610612740,NOP,New Orleans Pelicans,NOP @ NYK,W,241,117,41,84,0.488,17,41,0.415,18,33,0.545,12,33,45,21,6,3,21,23,1.0
3,22019,1610612752,NYK,New York Knicks,0021900029,2019-10-26,NYK vs. BOS,L,241,95,31,82,0.378,7,27,0.259,26,34,0.765,16,39,55,15,7,4,23,26,-23.0,1610612738,BOS,Boston Celtics,BOS @ NYK,W,242,118,42,93,0.452,15,41,0.366,19,29,0.655,13,33,46,24,14,9,12,26,23.0
4,22019,1610612752,NYK,New York Knicks,0021900042,2019-10-28,NYK vs. CHI,W,241,105,38,91,0.418,9,29,0.310,20,32,0.625,25,38,63,19,3,6,17,19,7.0,1610612741,CHI,Chicago Bulls,CHI @ NYK,L,240,98,36,85,0.424,9,35,0.257,17,21,0.810,8,30,38,19,6,5,9,22,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,22023,1610612752,NYK,New York Knicks,0022301082,2024-03-31,NYK vs. OKC,L,240,112,38,92,0.413,13,43,0.302,23,34,0.676,15,31,46,24,8,3,10,19,-1.0,1610612760,OKC,Oklahoma City Thunder,OKC @ NYK,W,241,113,44,82,0.537,13,34,0.382,12,24,0.500,9,34,43,29,6,8,12,23,1.0
212,22023,1610612752,NYK,New York Knicks,0022301112,2024-04-04,NYK vs. SAC,W,241,120,44,80,0.550,12,28,0.429,20,29,0.690,9,30,39,34,9,2,11,20,11.0,1610612758,SAC,Sacramento Kings,SAC @ NYK,L,241,109,42,86,0.488,18,42,0.429,7,12,0.583,9,26,35,26,6,3,16,23,-11.0
213,22023,1610612752,NYK,New York Knicks,0022301175,2024-04-12,NYK vs. BKN,W,241,111,38,83,0.458,14,30,0.467,21,25,0.840,8,23,31,21,10,3,5,15,4.0,1610612751,BKN,Brooklyn Nets,BKN @ NYK,L,241,107,41,82,0.500,11,32,0.344,14,19,0.737,16,29,45,22,3,4,12,19,-4.0
214,22023,1610612752,NYK,New York Knicks,0022301190,2024-04-14,NYK vs. CHI,W,265,120,46,91,0.505,12,32,0.375,16,21,0.762,16,37,53,27,7,6,21,17,1.0,1610612741,CHI,Chicago Bulls,CHI @ NYK,L,265,119,49,101,0.485,8,31,0.258,13,16,0.813,15,29,44,25,15,3,12,21,-1.0


In [108]:
def rolling_averages(group, cols, new_cols):
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [109]:
cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
away_cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY']
home_cols = ['PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
preds = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']

new_cols = [f"{c}_rolling" for c in cols]
new_cols_home = [f"{c}_rolling" for c in home_cols]
new_cols_home.append(home_cols[-1])
new_cols_away = [f"{c}_rolling" for c in away_cols]
new_cols_away.append(away_cols[-1])

pred_cols = new_cols_away+new_cols_home

preds = [f"{c}_rolling" for c in preds]

def add_new_cols(team_id):
    df_home, df_away = get_dfs(team_id)
    
    df_home = rolling_averages(df_home, cols, new_cols)
    df_away = rolling_averages(df_away, cols, new_cols)
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [110]:
def df_with_target(team_id):
    df_home, df_away = add_new_cols(team_id)
    df_home["W"] = (df_home["WL_HOME"] == "W").astype("int")
    df_away["W"] = (df_away["WL_AWAY"] == "W").astype("int")
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [111]:
df1, df2 = df_with_target(knicks)
df1

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,PTS_AWAY_rolling,FGM_AWAY_rolling,FGA_AWAY_rolling,FG_PCT_AWAY_rolling,FG3M_AWAY_rolling,FG3A_AWAY_rolling,FG3_PCT_AWAY_rolling,FTM_AWAY_rolling,FTA_AWAY_rolling,FT_PCT_AWAY_rolling,OREB_AWAY_rolling,DREB_AWAY_rolling,REB_AWAY_rolling,AST_AWAY_rolling,TOV_AWAY_rolling,PLUS_MINUS_AWAY_rolling,PTS_HOME_rolling,FGM_HOME_rolling,FGA_HOME_rolling,FG_PCT_HOME_rolling,FG3M_HOME_rolling,FG3A_HOME_rolling,FG3_PCT_HOME_rolling,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612765,DET,Detroit Pistons,0021900103,2019-11-06,DET vs. NYK,W,240,122,44,79,0.557,15,30,0.500,19,24,0.792,8,35,43,37,7,1,16,17,20.0,1610612752,NYK,New York Knicks,NYK @ DET,L,242,102,39,80,0.488,11,25,0.440,13,21,0.619,5,25,30,24,9,4,13,20,-20.0,106.2,36.6,84.0,0.4394,11.6,34.2,0.3438,21.4,27.0,0.7910,9.6,39.8,49.4,22.8,15.6,4.4,101.8,37.6,88.2,0.4262,12.6,31.8,0.4072,14.0,21.2,0.6688,9.6,36.0,45.6,21.4,15.8,-4.4,0
1,22019,1610612742,DAL,Dallas Mavericks,0021900122,2019-11-08,DAL vs. NYK,L,241,102,37,90,0.411,14,44,0.318,14,17,0.824,7,40,47,18,4,6,13,21,-4.0,1610612752,NYK,New York Knicks,NYK @ DAL,W,241,106,37,89,0.416,14,28,0.500,18,31,0.581,14,42,56,19,9,3,12,20,4.0,110.8,38.6,80.4,0.4806,12.2,30.8,0.3928,21.4,27.0,0.7910,9.2,38.0,47.2,25.0,16.8,9.4,101.4,37.8,86.0,0.4402,11.8,29.8,0.4094,14.0,21.6,0.6558,9.0,31.6,40.6,21.2,15.4,-9.4,1
2,22019,1610612741,CHI,Chicago Bulls,0021900149,2019-11-12,CHI vs. NYK,W,241,120,41,81,0.506,14,33,0.424,24,31,0.774,9,37,46,22,9,2,19,25,18.0,1610612752,NYK,New York Knicks,NYK @ CHI,L,241,102,36,88,0.409,10,32,0.313,20,31,0.645,13,25,38,27,12,1,18,28,-18.0,107.2,37.6,81.8,0.4616,13.6,35.4,0.3898,18.4,23.2,0.7946,8.6,37.8,46.4,24.2,15.2,6.8,100.4,36.4,85.0,0.4298,12.6,29.4,0.4428,15.0,24.2,0.6276,10.8,33.2,44.0,20.2,15.0,-6.8,0
3,22019,1610612755,PHI,Philadelphia 76ers,0021900202,2019-11-20,PHI vs. NYK,W,240,109,39,89,0.438,13,32,0.406,18,26,0.692,10,34,44,30,5,5,10,14,5.0,1610612752,NYK,New York Knicks,NYK @ PHI,L,241,104,40,84,0.476,13,25,0.520,11,15,0.733,7,35,42,17,8,6,14,25,-5.0,108.6,38.4,82.4,0.4680,13.2,34.6,0.3882,18.6,22.6,0.8142,8.2,38.2,46.4,24.4,13.8,9.6,99.0,36.0,86.4,0.4178,11.2,30.8,0.3694,15.8,24.8,0.6424,11.6,32.2,43.8,22.2,13.4,-9.6,0
4,22019,1610612761,TOR,Toronto Raptors,0021900258,2019-11-27,TOR vs. NYK,W,240,126,46,94,0.489,21,41,0.512,13,17,0.765,17,35,52,26,6,4,12,18,28.0,1610612752,NYK,New York Knicks,NYK @ TOR,L,243,98,33,88,0.375,12,41,0.293,20,24,0.833,15,30,45,19,6,4,18,17,-28.0,111.4,38.8,82.6,0.4716,13.4,34.6,0.3944,20.4,25.4,0.8026,7.4,37.4,44.8,25.6,14.4,8.2,103.2,37.8,86.8,0.4374,12.4,29.6,0.4282,15.2,23.8,0.6490,11.0,32.4,43.4,21.8,14.0,-8.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22023,1610612748,MIA,Miami Heat,0022301096,2024-04-02,MIA vs. NYK,W,241,109,36,74,0.486,17,38,0.447,20,21,0.952,7,32,39,24,5,4,17,17,10.0,1610612752,NYK,New York Knicks,NYK @ MIA,L,239,99,35,76,0.461,12,34,0.353,17,21,0.810,7,23,30,25,11,2,14,17,-10.0,109.4,40.0,84.6,0.4734,11.4,32.8,0.3428,18.0,21.8,0.8344,10.2,29.2,39.

In [112]:
#combines home games of all teams and away games of all teams
all_home, all_away = df_with_target(cavs)
    
for team in no_cavs:
    df_home, df_away = df_with_target(team)
    all_home = all_home.append(df_home)
    all_away = all_away.append(df_away)
    
all_home.index = range(0,all_home.shape[0])
all_away.index = range(0,all_away.shape[0])
all_home = all_home.dropna()
all_away = all_away.dropna()
        
all_home

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,PTS_AWAY_rolling,FGM_AWAY_rolling,FGA_AWAY_rolling,FG_PCT_AWAY_rolling,FG3M_AWAY_rolling,FG3A_AWAY_rolling,FG3_PCT_AWAY_rolling,FTM_AWAY_rolling,FTA_AWAY_rolling,FT_PCT_AWAY_rolling,OREB_AWAY_rolling,DREB_AWAY_rolling,REB_AWAY_rolling,AST_AWAY_rolling,TOV_AWAY_rolling,PLUS_MINUS_AWAY_rolling,PTS_HOME_rolling,FGM_HOME_rolling,FGA_HOME_rolling,FG_PCT_HOME_rolling,FG3M_HOME_rolling,FG3A_HOME_rolling,FG3_PCT_HOME_rolling,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612764,WAS,Washington Wizards,0021900118,2019-11-08,WAS vs. CLE,L,239,100,45,96,0.469,7,31,0.226,3,6,0.500,17,30,47,26,5,5,18,24,-13.0,1610612739,CLE,Cleveland Cavaliers,CLE @ WAS,W,240,113,41,90,0.456,6,27,0.222,25,29,0.862,17,31,48,18,10,5,12,17,13.0,110.4,42.2,89.8,0.4702,9.6,29.4,0.3094,16.4,23.4,0.6968,9.8,40.0,49.8,25.2,14.6,16.6,93.8,34.6,88.0,0.3904,10.2,35.8,0.2890,14.4,18.8,0.7628,8.2,37.6,45.8,17.4,17.4,-16.6,1
1,22019,1610612752,NYK,New York Knicks,0021900136,2019-11-10,NYK vs. CLE,L,239,87,31,86,0.360,9,31,0.290,16,25,0.640,17,35,52,19,11,8,21,26,-21.0,1610612739,CLE,Cleveland Cavaliers,CLE @ NYK,W,240,108,36,82,0.439,8,28,0.286,28,31,0.903,9,36,45,20,11,2,14,19,21.0,108.6,43.0,92.0,0.4676,10.2,31.2,0.3182,12.4,18.4,0.6484,11.8,38.2,50.0,25.4,14.8,13.2,95.4,35.4,88.8,0.3956,9.0,34.4,0.2628,15.6,19.8,0.7768,10.0,37.4,47.4,17.6,16.4,-13.2,1
2,22019,1610612755,PHI,Philadelphia 76ers,0021900147,2019-11-12,PHI vs. CLE,W,240,98,40,89,0.449,8,38,0.211,10,13,0.769,11,35,46,24,8,8,14,20,1.0,1610612739,CLE,Cleveland Cavaliers,CLE @ PHI,L,239,97,38,86,0.442,5,21,0.238,16,20,0.800,9,36,45,13,10,0,13,13,-1.0,102.4,40.0,91.2,0.4374,9.4,31.4,0.2896,13.0,19.0,0.6582,12.4,38.2,50.6,23.6,15.4,-0.2,102.6,37.4,89.4,0.4176,9.2,32.0,0.2850,18.6,22.4,0.8130,9.8,38.2,48.0,19.4,13.8,0.2,0
3,22019,1610612752,NYK,New York Knicks,0021900189,2019-11-18,NYK vs. CLE,W,241,123,44,98,0.449,8,23,0.348,27,35,0.771,19,28,47,17,8,10,7,24,18.0,1610612739,CLE,Cleveland Cavaliers,CLE @ NYK,L,240,105,36,76,0.474,17,36,0.472,16,30,0.533,10,31,41,18,2,1,19,26,-18.0,103.2,40.6,91.8,0.4412,9.2,33.0,0.2718,12.8,17.8,0.6962,13.2,37.4,50.6,23.6,15.6,-1.8,105.0,38.4,89.0,0.4310,8.4,29.4,0.2796,19.8,23.4,0.8396,9.6,36.6,46.2,18.0,13.2,1.8,0
4,22019,1610612748,MIA,Miami Heat,0021900206,2019-11-20,MIA vs. CLE,W,238,124,41,78,0.526,19,37,0.514,23,31,0.742,12,32,44,25,8,5,23,20,24.0,1610612739,CLE,Cleveland Cavaliers,CLE @ MIA,L,239,100,37,87,0.425,12,38,0.316,14,16,0.875,10,24,34,19,13,2,19,23,-24.0,102.0,39.8,93.0,0.4266,7.4,30.0,0.2520,15.0,20.0,0.7170,15.4,34.6,50.0,21.2,14.6,-1.6,103.6,36.6,84.2,0.4358,9.4,28.8,0.3124,21.0,27.0,0.7796,9.6,35.2,44.8,17.8,14.8,1.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6587,22023,1610612741,CHI,Chicago Bulls,0022300971,2024-03-16,CHI vs. WAS,W,240,127,49,85,0.576,8,24,0.333,21,29,0.724,12,39,51,33,9,4,15,16,29.0,1610612764,WAS,Washington Wizards,WAS @ CHI,L,241,98,36,84,0.429,13,32,0.406,13,22,0.591,9,23,32,27,12,7,15,21,-29.0,123.8,47.8,94.2,0.5090,11.8,33.2,0.3560,16.4,20.6

In [159]:
model = GaussianNB()

home_train, home_test = train_test_split(all_home, test_size=0.25)
model_home = model.fit(home_train[pred_cols], home_train["W"])
preds = model.predict(home_test[pred_cols])
combined_home = pd.DataFrame(dict(actual=home_test["W"], predicted=preds), index=home_test.index)
accuracy_home = accuracy_score(home_test["W"], preds)
precision_home = precision_score(home_test["W"], preds)

away_train, away_test = train_test_split(all_away, test_size=0.25)
model_away = model.fit(away_train[pred_cols], away_train["W"])
preds = model.predict(away_test[pred_cols])
combined_away = pd.DataFrame(dict(actual=away_test["W"], predicted=preds), index=away_test.index)
accuracy_away = accuracy_score(away_test["W"], preds)
precision_away = precision_score(away_test["W"], preds)

print("Home")
print("Accuracy: " + str(accuracy_home))
print("Precision: " + str(precision_home))
print()
print("Away")
print("Accuracy: " + str(accuracy_away))
print("Precision: " + str(precision_away))

combined_away

Home
Accuracy: 0.970873786407767
Precision: 0.9629629629629629

Away
Accuracy: 0.9486714975845411
Precision: 0.956953642384106


,actual,predicted
4271,1,1
5570,0,0
880,0,0
5430,1,1
3365,0,0
...,...,...
3449,1,1
2006,0,0
6190,0,0
9,0,0


In [135]:
def predictor(away_id, home_id):
    df_home, df_away1 = df_with_target(home_id)
    df_home1, df_away = df_with_target(away_id)
    
    home_stats = list(df_home.loc[:, 'PTS_HOME_rolling':'PLUS_MINUS_HOME_rolling'].iloc[-1])
    home_pm = df_home['PLUS_MINUS_HOME'].iloc[-1]
    home_stats.append(home_pm)
    away_stats = list(df_away.loc[:, 'PTS_AWAY_rolling':'PLUS_MINUS_AWAY_rolling'].iloc[-1])
    away_pm = df_away['PLUS_MINUS_AWAY'].iloc[-1]
    away_stats.append(away_pm)
    new_stats = [away_stats+home_stats]
        
    df_predict = pd.DataFrame(new_stats, columns=pred_cols)
    df_predict['W'] = model_away.predict(new_stats)
    
    if df_predict['W'].iloc[0] == 1:
        result = "win"
    else: 
        result = "lose"
        
    print(f"The away team will {result} against the home team.")

In [105]:
predictor(sixers, knicks)
predictor(magic, cavs)
predictor(lakers, nuggets)

The away team will lose against the home team.
The away team will win against the home team.
The away team will lose against the home team.
